# Demo notebook for Kamodo's Reconstruction: Spherical Coordinates
### The short version
This notebook tutorial shows how to fly a constellation of satellites through model data as a virtual reality, focusing on spherical coordinate examples. Trajectories can be obtained either from the test GDC trajectory file or the flythrough trajectory functions as shown here. See the Trajectory_Coord_Plots demo notebook for examples on flythrough trajectory options, and the START_HERE notebook for more information on choosing models and variables.

In [ ]:
# For a given model, find out what time range is covered by the data in a given directory.
from kamodo_ccmc.flythrough import model_wrapper as MW
model, file_dir = 'GITM', 'D:/GITM/jasoon_shim_071418_IT_1_tenth/'  # change to match your machine
times = MW.File_Times(model, file_dir)
times[0].timestamp(), times[1].timestamp()
# This function also automatically performs any data preparation needed.

In [ ]:
# Import function to retrieve the grace1 trajectory from the SSCWeb.
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
# Typical coordinates possible through SSCWeb are GEO, GSE, SM, and GSM (all cartesian and in R_E).
input_coord = 'GEO'
traj_dict, coord_type = SF.SatelliteTrajectory('grace1', times[0].timestamp(), times[1].timestamp(), coord_type=input_coord)

In [ ]:
# Convert coordinates to system desired for reconstruction to take place in.
# The trajectories were retrieved in GEO cartesian, and are converted to GEO spherical below.
from kamodo_ccmc.flythrough.utils import ConvertCoord
c1, c2, c3, units = ConvertCoord(traj_dict['sat_time'], traj_dict['c1'], traj_dict['c2'], traj_dict['c3'],
                                 input_coord, 'car', input_coord, 'sph')
print(c1.min(), c1.max(), c2.min(), c2.max(), c3.min(), c3.max())

In [ ]:
# Choose a variable from the model data chosen.
MW.Variable_Search('', model, file_dir)

In [ ]:
# Bring up documentation for the reconstruction functionality.
from kamodo_ccmc.flythrough.Reconstruction_v0 import RECON
help(RECON)

In [ ]:
# Choose inputs.
variable_name = 'rho_n'  # from chosen files above
recon_dimensions = 'c1c2'  # Longitude vs Latitude reconstruction for spherical coordinates
recon_option = 'UnMod_AvgDSlice'
# fly the given trajectory through the data unmodified, then fly the reconstructed coordinate grid through the data
# after taking the average of t and height.

# Set up constellation input:
lon_offsets = [0., 30., 60., 90., 120., 150.]  # 6 satellites equally spaced in longitude

# Choose the grid resolution of reconstruction. The finer the resolution, the longer the program takes to run and 
#   the more 'holes' you will see in the reconstructed plot. Physically, these should be set to the instrument's 
#   field of view in the units of the input coordinate system (e.g. degrees for longitude and latitude, seconds for
#   time, etc).
dx, dy = 5., 5.  # Since recon_dimensions='c1c2', dx is resolution in longitude, and dy is the resolution in latitude.
d1, d2 = 1800., 0.001  # time (in s) and height (in R_E) resolution of sampling for averaging

In [ ]:
# Run the reconstruction.
# This process typically takes up to a few minutes, but can take up to ~2 hours or more depending on the amount of data 
#  used, the grid resolution chosen, the reconstruction method chosen, and whether conversion to pressure level is 
#  required. As originally set, the process takes about 360 seconds.
# Make sure to include all desired offsets in this block before executing.

recon = RECON(model, variable_name, file_dir, traj_dict['sat_time'], c1, c2, c3,
              input_coord, 'sph', recon_option, recon_dimensions, c1_offsets=lon_offsets, dx=dx, dy=dy, d1=d1, d2=d2)

In [ ]:
# The output of the function is a Kamodo object with all of the default features described in documentation.
# rho_n is the reconstructed data from the constellation flythrough. rho_n_model is the data from the model in the 
#  method chosen. PercentDiff is the percent difference between the two, calculated using 
#  PercentDiff = (rho_n_model - rho_n)/rho_n_model*100.
recon

In [ ]:
# Nicer version of default Kamodo plot
fig = recon.plot(rho_n=dict(Lon=recon.x, Lat=recon.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
fig = recon.plot(rho_n_model=dict(Lon=recon.x, Lat=recon.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Show the percent difference between the two. A percent difference of zero is an exact match.
# How well the reconstructed plot matches the model plot not only depends on the constellation arrangement,
#  but also on the reconstruction method chosen. The 'AvgMod_...' options typically result in the better
#  matches, but are not physically representative of what the constellation will 'see' in real data because
#  the two non-reconstructed dimensions are ignored in the input satellite trajectory (e.g. an average value 
#  for both height and time are used instead of the full range for a Lon-Lat reconstruction). The unmodified 
#  options ('Unmod_...') are thus recommended as the more physical comparison because the full set of input 
#  trajectory values are used.
fig = recon.plot(PercentDiff=dict(Lon=recon.x, Lat=recon.y))
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Retrieve the percent difference data values and show in a histogram, ignoring NaN values.
# Some extra logic is required to automatically enforce bins of width 2%
import numpy as np
import matplotlib.pyplot as plt

pdiff_data = recon.PercentDiff()
data_min, data_max = np.floor(np.nanmin(pdiff_data)), np.ceil(np.nanmax(pdiff_data))
num_bins = int((data_max-data_min)/2.)
if num_bins<5:
    num_bins=20
hist, edges, patches = plt.hist(np.ravel(pdiff_data), range=(data_min,data_max),bins=num_bins)
plt.xlabel('Percent Difference')
plt.ylabel('Frequency')